In [2]:
from nba_api.stats.endpoints import leagueleaders
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from nba_api.stats.endpoints import leaguedashplayerstats, synergyplaytypes, leaguedashteamstats
from nba_api.stats.endpoints import teamyearbyyearstats, playercareerstats
from nba_api.stats.static.players import find_players_by_full_name, find_player_by_id
from nba_api.stats.static.teams import find_team_name_by_id
from matplotlib.offsetbox import OffsetImage, AnnotationBbox, DrawingArea, TextArea
from matplotlib.patches import Circle, FancyArrowPatch, FancyArrow, ArrowStyle, Arrow, Rectangle
import matplotlib.ticker as mtick
import time
from tqdm import tqdm
from sklearn.cluster import KMeans
import numpy as np
import functools
import seaborn as sns
import glob
from sklearn import preprocessing
from scipy import stats
from fitter import Fitter, get_common_distributions, get_distributions
import sklearn
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import gamma, exponpow, lognorm, cauchy, genhyperbolic
from datetime import date
from PIL import Image
from collections import defaultdict
import re


pd.set_option('display.max_columns', None)
# pd.reset_option(‘max_columns’)

In [3]:
# EVENTMSGACTIONTYPE
# ---
# 1 - Jumpshot
# 2 - Lost ball Turnover
# 3 - ?
# 4 - Traveling Turnover / Off Foul
# 5 - Layup
# 7 - Dunk
# 10 - Free throw 1-1
# 11 - Free throw 1-2
# 12 - Free throw 2-2
# 40 - out of bounds
# 41 - Block/Steal
# 42 - Driving Layup
# 50 - Running Dunk
# 52 - Alley Oop Dunk
# 55 - Hook Shot
# 57 - Driving Hook Shot
# 58 - Turnaround hook shot
# 66 - Jump Bank Shot
# 71 - Finger Roll Layup
# 72 - Putback Layup
# 108 - Cutting Dunk Shot

In [4]:
# EVENTMSGTYPE
# ---
# 1 - Make
# 2 - Miss
# 3 - Free Throw
# 4 - Rebound
# 5 - out of bounds / Turnover / Steal
# 6 - Personal Foul
# 7 - Violation
# 8 - Substitution
# 9 - Timeout
# 10 - Jumpball
# 12 - Start Q1?
# 13 - Start Q2?

In [5]:
df = pd.read_csv('../utils/pbp_data/2023-24.csv')

In [25]:
turnovers = df[(df['EVENTMSGTYPE'] == 5)]

turnovers.head(30)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,PCTIMESECONDS
4,22301198,10,5,6,1,3:43 PM,11:02,NaN,NaN,Yurtseven Double Dribble Turnover (P1.T1),NaN,NaN,5,1630209,Omer Yurtseven,1.610613e+09,Utah,Jazz,UTA,0,0,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,1,58
16,22301198,23,5,1,1,3:45 PM,9:42,Wiggins STEAL (1 STL),NaN,George Bad Pass Turnover (P1.T2),NaN,NaN,5,1641718,Keyonte George,1.610613e+09,Utah,Jazz,UTA,4,203952,Andrew Wiggins,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,1,138
17,22301198,26,5,1,1,3:45 PM,9:34,Wiggins Bad Pass Turnover (P1.T1),NaN,Samanic STEAL (1 STL),NaN,NaN,4,203952,Andrew Wiggins,1.610613e+09,Golden State,Warriors,GSW,5,1629677,Luka Samanic,1.610613e+09,Utah,Jazz,UTA,0,0,NaN,NaN,NaN,NaN,NaN,1,146
20,22301198,31,5,4,1,3:46 PM,9:17,NaN,NaN,Samanic Traveling Turnover (P1.T3),NaN,NaN,5,1629677,Luka Samanic,1.610613e+09,Utah,Jazz,UTA,0,0,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,1,163
29,22301198,43,5,37,1,3:47 PM,7:51,NaN,NaN,Samanic Offensive Foul Turnover (P2.T4),NaN,NaN,5,1629677,Luka Samanic,1.610613e+09,Utah,Jazz,UTA,0,0,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,1,249
63,22301198,191,5,2,1,3:56 PM,3:16,Moody STEAL (1 STL),NaN,Samanic Lost Ball Turnover (P3.T5),NaN,NaN,5,1629677,Luka Samanic,1.610613e+09,Utah,Jazz,UTA,4,1630541,Moses Moody,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,1,524
101,22301198,152,5,1,2,4:11 PM,11:10,Saric Bad Pass Turnover (P1.T2),NaN,Sensabaugh STEAL (1 STL),NaN,NaN,4,203967,Dario Saric,1.610613e+09,Golden State,Warriors,GSW,5,1641729,Brice Sensabaugh,1.610613e+09,Utah,Jazz,UTA,0,0,NaN,NaN,NaN,NaN,NaN,1,50
104,22301198,157,5,4,2,4:11 PM,10:52,Moody Traveling Turnover (P1.T3),NaN,NaN,NaN,NaN,4,1630541,Moses Moody,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,1,68
105,22301198,158,5,2,2,4:11 PM,10:43,Podziemski STEAL (1 STL),NaN,Lofton Jr. Lost Ball Turnover (P1.T6),NaN,NaN,5,1631254,Kenneth Lofton Jr.,1.610613e+09,Utah,Jazz,UTA,4,1641764,Brandin Podziemski,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,1,77
125,22301198,185,5,1,2,4:17 PM,8:08,Podziemski STEAL (2 STL),NaN,Sensabaugh Bad Pass Turnover (P1.T7),NaN,NaN,5,1641729,Brice Sensabaugh,1.610613e+09,Utah,Jazz,UTA,4,1641764,Brandin Podziemski,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,1,232


In [23]:
def get_description(row):
    if pd.isna(row.HOMEDESCRIPTION):
        return row.VISITORDESCRIPTION
    else:
        return row.HOMEDESCRIPTION
    
def parse_shot_description(text):
    pattern = r"\b3PT\b"
    if re.search(pattern, text):
        return '3P'
    else:
        return '2P'
    

def check_next_shot_opposing_team(df, window_size=5):
    turnovers = df[(df['EVENTMSGTYPE'] == 5)  & (df['PLAYER2_ID'] != 0)]
    turnovers_idx = turnovers.index
    
    results = defaultdict(int)
    # new_df = pd.DataFrame()

    for idx in tqdm(turnovers_idx):
        window_end = min(idx + window_size, len(df))
        window = df.iloc[idx+1:window_end]
        
        turnover_team_id = df.loc[idx, 'PLAYER1_TEAM_ID']  # Team ID of the player committing the turnover
        next_team_id = df.loc[idx, 'PLAYER2_TEAM_ID']  # Team ID of the opposing team
        
        for i in range(idx+1, window_end):
            next_action = df.iloc[i]
            if next_action.PLAYER1_TEAM_ID == turnover_team_id:
                break
            else:
                next_action_type = next_action.EVENTMSGTYPE
                # the shot was shot from the opposing team
                if next_action_type == 1 or next_action_type == 2:
                    shot_desc = get_description(next_action)
                    shot_desc = parse_shot_description(shot_desc)
                    if next_action_type == 1:
                        shot_desc = f'{shot_desc}_MADE'
                        results[shot_desc] += 1
                        break
                    else:
                        shot_desc = f'{shot_desc}_MISS'
                        results[shot_desc] += 1
                else:
                    continue
    
    return results

In [15]:
results = check_next_shot_opposing_team(df, window_size=10)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15081/15081 [00:01<00:00, 12181.64it/s]


In [22]:
results

defaultdict(int,
            {'2P_MISS': 3613,
             '2P_MADE': 3923,
             '3P_MISS': 3007,
             '3P_MADE': 1721})

In [17]:
# EFG = FG + 0.5 * 3P) / FGA

In [18]:
shots_after_turnover = results['2P_MISS'] + results['3P_MISS'] + results['2P_MADE'] + results['3P_MADE']
made_shots_after_turnover = results['2P_MADE'] + results['3P_MADE']
twos_attemps = results['2P_MISS'] + results['2P_MADE']
threes_attemps = results['3P_MISS'] + results['3P_MADE']
twos_made = results['2P_MADE']
threes_made = results['3P_MADE']

fg = made_shots_after_turnover / shots_after_turnover

print(f'FG  -\t{np.round((fg * 100), 2)}%')

efg = (results['2P_MADE'] + results['3P_MADE'] + 0.5 * results['3P_MADE']) / shots_after_turnover

print(f'EFG -\t{np.round((efg * 100), 2)}%')

threes_perc = threes_made / threes_attemps 
twos_perc = twos_made / twos_attemps
threes_perc *= 100
twos_perc *= 100

print(f'2P% - \t{np.round(twos_perc, 2)}%')
print(f'3P% - \t{np.round(threes_perc, 2)}%')


FG  -	46.02%
EFG -	53.04%
2P% - 	52.06%
3P% - 	36.4%


In [297]:
shots_after_turnover

27954